In [54]:
import numpy as np

from cachesim import CacheSimulator, Cache, MainMemory
from argparse import ArgumentParser


In [55]:
def make_cache() -> CacheSimulator:
    mem = MainMemory()
    l3 = Cache("L3", 20480, 16, 64, "LRU")
    mem.load_to(l3)
    mem.store_from(l3)
    l2 = Cache("L2", 512, 8, 64, "LRU", store_to=l3, load_from=l3)
    l1 = Cache("L1", 64, 8, 64, "LRU", store_to=l2, load_from=l2)
    cs = CacheSimulator(l1, mem)
    return cs


algorithm, N, K = 'recursive', 8, 4

cs1 = make_cache()
cs2 = make_cache()

rnd_vals1 = np.random.rand(N, N)
rnd_vals2 = np.random.rand(N, N)

In [56]:
# block index to row-major index
def normalize_index(i, j, N, K) -> tuple:
    block_row = i // K
    inside_block_row = i % K

    block_column = j // K
    inside_block_column = j % K

    row_block_size = N * K
    column_block_size = K * K

    idx = block_row * row_block_size + block_column * column_block_size + inside_block_row * K + inside_block_column

    normalized_i = idx // N
    normalized_j = idx % N

    return (normalized_i, normalized_j)

In [57]:
def fill_row_major_arrays(A, B, N, cs):
  for i in range(N):
    for j in range(N):
        idx = i * N + j

        A[i][j] = rnd_vals1[i][j]
        cs.store(idx) #store operation at position i * N + j in row major to memory(A1)

        B[i][j] = rnd_vals2[i][j]
        cs.store(N * N + idx) #store operation at position N * N + i * N + j in row major to memory(B1)


def fill_block_arrays(A, B, N, cs):
  for i in range(N):
    for j in range(N):
        (normalized_i, normalized_j) = normalize_index(i, j, N, K)

        A[normalized_i][normalized_j] = rnd_vals1[i][j]
        print("i: " + str(i) + " j: " + str(j) + " normalized_i: " + str(normalized_i) + " normalized_j: " + str(normalized_j) + " idx: " + str(normalized_i * N + normalized_j))
        cs.store(normalized_i * N + normalized_j) #store operation at position i * N + j in row major to memory(A1)

        B[normalized_i][normalized_j] = rnd_vals2[i][j]
        cs.store(N * N + normalized_i * N + normalized_j) #store operation at position N * N + i * N + j in row major to memory(B1)

In [58]:
# Test block
# create array of shape
A = np.zeros((8, 8))
B = np.zeros((8, 8))
rnd_vals1 = np.arange(64).reshape(8, 8)
rnd_vals2 = np.arange(64).reshape(8, 8)
fill_block_arrays(A, B, 8, cs1)
# Test block end

i: 0 j: 0 normalized_i: 0 normalized_j: 0 idx: 0
i: 0 j: 1 normalized_i: 0 normalized_j: 1 idx: 1
i: 0 j: 2 normalized_i: 0 normalized_j: 2 idx: 2
i: 0 j: 3 normalized_i: 0 normalized_j: 3 idx: 3
i: 0 j: 4 normalized_i: 2 normalized_j: 0 idx: 16
i: 0 j: 5 normalized_i: 2 normalized_j: 1 idx: 17
i: 0 j: 6 normalized_i: 2 normalized_j: 2 idx: 18
i: 0 j: 7 normalized_i: 2 normalized_j: 3 idx: 19
i: 1 j: 0 normalized_i: 0 normalized_j: 4 idx: 4
i: 1 j: 1 normalized_i: 0 normalized_j: 5 idx: 5
i: 1 j: 2 normalized_i: 0 normalized_j: 6 idx: 6
i: 1 j: 3 normalized_i: 0 normalized_j: 7 idx: 7
i: 1 j: 4 normalized_i: 2 normalized_j: 4 idx: 20
i: 1 j: 5 normalized_i: 2 normalized_j: 5 idx: 21
i: 1 j: 6 normalized_i: 2 normalized_j: 6 idx: 22
i: 1 j: 7 normalized_i: 2 normalized_j: 7 idx: 23
i: 2 j: 0 normalized_i: 1 normalized_j: 0 idx: 8
i: 2 j: 1 normalized_i: 1 normalized_j: 1 idx: 9
i: 2 j: 2 normalized_i: 1 normalized_j: 2 idx: 10
i: 2 j: 3 normalized_i: 1 normalized_j: 3 idx: 11
i: 2 j: 4 

In [59]:
def matrix_multiply_row_major(A, B, C, N, cs):

  for i in range(N):
    for j in range(N):
        for k in range(N):
            mul1 = A[i, k]
            cs.load(N * i + k) #load operation at position N * i + k to row major cache

            mul2 = B[k, j]
            cs.load(N * k + j) #load operation at position N * k + j to row major cache

            C[i, j] += mul1 * mul2
            cs.store(2 * N * N + N * i + j) #store operation at position 2 * N * N + N * i + j in row major to memory(C1)


def matrix_multiply_blocks(A, B, C, N, cs):
  for i in range(N):
    for j in range(N):
        for k in range(N):
            (n_i, n_k) = normalize_index(i, k, N, K)
            mul1 = A[n_i, n_k]
            cs.load(N * n_i + n_k) #load operation at position N * i + k to row major cache

            (n_k, n_j) = normalize_index(k, j, N, K)
            mul2 = B[n_k, n_j]
            cs.load(N * n_k + n_j) #load operation at position N * k + j to row major cache

            C[i, j] += mul1 * mul2
            cs.store(2 * N * N + N * n_i + n_j) #store operation at position 2 * N * N + N * i + j in row major to memory(C2)


In [60]:
def recursive_matrix_multiply_row_major(A, B, result, i, j, k, n, cs):
    if n == 1:
        mul1 = A[i, k]
        cs.load(n * i + k) #load operation at position N * i + k to row major cache

        mul2 = B[k, j]
        cs.load(n * k + j) #load operation at position N * k + j to row major cache

        result[i, j] += mul1 * mul2
        cs.store(2 * n * n + n * i + j) #store operation at position 2 * N * N + N * i + j in row major to memory(C1)
        return

    mid = n // 2

    # Recursive matrix multiplications
    recursive_matrix_multiply_row_major(A, B, result, i, j, k, mid, cs)
    recursive_matrix_multiply_row_major(A, B, result, i, j, k + mid, mid, cs)

    recursive_matrix_multiply_row_major(A, B, result, i, j + mid, k, mid, cs)
    recursive_matrix_multiply_row_major(A, B, result, i, j + mid, k + mid, mid, cs)

    recursive_matrix_multiply_row_major(A, B, result, i + mid, j, k, mid, cs)
    recursive_matrix_multiply_row_major(A, B, result, i + mid, j, k + mid, mid, cs)

    recursive_matrix_multiply_row_major(A, B, result, i + mid, j + mid, k, mid, cs)
    recursive_matrix_multiply_row_major(A, B, result, i + mid, j + mid, k + mid, mid, cs)


def recursive_matrix_multiply_blocks(A, B, result, i, j, k, n, cs):
    if n == 1:
        print(f"A[i {i}, k {k}] , B[k {k}, j {j}] => C[i {i}, j {j}]")
        (n_i, n_k) = normalize_index(i, k, N, K)
        mul1 = A[n_i, n_k]
        cs.load(n * n_i + n_k) #load operation at position N * i + k to row major cache

        (n_k, n_j) = normalize_index(k, j, N, K)
        mul2 = B[n_k, n_j]
        cs.load(n * n_k + n_j) #load operation at position N * k + j to row major cache

        result[i, j] += mul1 * mul2
        cs.store(2 * n * n + n * n_i + n_j) #store operation at position 2 * N * N + N * i + j in row major to memory(C1)
        return

    mid = n // 2

    # Recursive matrix multiplications
    recursive_matrix_multiply_blocks(A, B, result, i, j, k, mid, cs)
    recursive_matrix_multiply_blocks(A, B, result, i, j, k + mid, mid, cs)

    recursive_matrix_multiply_blocks(A, B, result, i, j + mid, k, mid, cs)
    recursive_matrix_multiply_blocks(A, B, result, i, j + mid, k + mid, mid, cs)

    recursive_matrix_multiply_blocks(A, B, result, i + mid, j, k, mid, cs)
    recursive_matrix_multiply_blocks(A, B, result, i + mid, j, k + mid, mid, cs)

    recursive_matrix_multiply_blocks(A, B, result, i + mid, j + mid, k, mid, cs)
    recursive_matrix_multiply_blocks(A, B, result, i + mid, j + mid, k + mid, mid, cs)
   


In [61]:
# WRITE YOUR CODE BELOW #
print("Given Array 1: ")
print(rnd_vals1)
print()
print("Given Array 2: ")
print(rnd_vals2)
print()

A1 = np.zeros([N, N])
B1 = np.zeros([N, N])
C1 = np.zeros([N, N])

fill_row_major_arrays(A1, B1, N, cs1)

if (algorithm == 'simple'):
  matrix_multiply_row_major(A1, B1, C1, N, cs1)
elif (algorithm == 'recursive'):
  recursive_matrix_multiply_row_major(A1, B1, C1, 0, 0, 0, N, cs1)

print("Row major result: ")
print(C1)
print()

A2 = np.zeros([N, N])
B2 = np.zeros([N, N])
C2 = np.zeros([N, N])

fill_block_arrays(A2, B2, N, cs2)

if (algorithm == 'simple'):
  matrix_multiply_blocks(A2, B2, C2, N, cs2)
elif (algorithm == 'recursive'):
  recursive_matrix_multiply_blocks(A2, B2, C2, 0, 0, 0, N, cs2)

print("block result: ")
print(C2)


# WRITE YOUR CODE ABOVE #
print('Row major array')
cs1.print_stats()


print('Block array')
cs2.print_stats()

Given Array 1: 
[[ 0  1  2  3  4  5  6  7]
 [ 8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23]
 [24 25 26 27 28 29 30 31]
 [32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47]
 [48 49 50 51 52 53 54 55]
 [56 57 58 59 60 61 62 63]]

Given Array 2: 
[[ 0  1  2  3  4  5  6  7]
 [ 8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23]
 [24 25 26 27 28 29 30 31]
 [32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47]
 [48 49 50 51 52 53 54 55]
 [56 57 58 59 60 61 62 63]]

Row major result: 
[[ 1120.  1148.  1176.  1204.  1232.  1260.  1288.  1316.]
 [ 2912.  3004.  3096.  3188.  3280.  3372.  3464.  3556.]
 [ 4704.  4860.  5016.  5172.  5328.  5484.  5640.  5796.]
 [ 6496.  6716.  6936.  7156.  7376.  7596.  7816.  8036.]
 [ 8288.  8572.  8856.  9140.  9424.  9708.  9992. 10276.]
 [10080. 10428. 10776. 11124. 11472. 11820. 12168. 12516.]
 [11872. 12284. 12696. 13108. 13520. 13932. 14344. 14756.]
 [13664. 14140. 14616. 15092. 15568. 16044. 16520. 16996.]]

i: 0 j: 0 normalized_i: 0 normalized_j:

In [62]:
(C1==C2).all()

True

In [63]:
C2

array([[ 1120.,  1148.,  1176.,  1204.,  1232.,  1260.,  1288.,  1316.],
       [ 2912.,  3004.,  3096.,  3188.,  3280.,  3372.,  3464.,  3556.],
       [ 4704.,  4860.,  5016.,  5172.,  5328.,  5484.,  5640.,  5796.],
       [ 6496.,  6716.,  6936.,  7156.,  7376.,  7596.,  7816.,  8036.],
       [ 8288.,  8572.,  8856.,  9140.,  9424.,  9708.,  9992., 10276.],
       [10080., 10428., 10776., 11124., 11472., 11820., 12168., 12516.],
       [11872., 12284., 12696., 13108., 13520., 13932., 14344., 14756.],
       [13664., 14140., 14616., 15092., 15568., 16044., 16520., 16996.]])

In [67]:
normalize_index(0, 4, 8, 4)

(2, 0)

In [65]:
A1

array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11., 12., 13., 14., 15.],
       [16., 17., 18., 19., 20., 21., 22., 23.],
       [24., 25., 26., 27., 28., 29., 30., 31.],
       [32., 33., 34., 35., 36., 37., 38., 39.],
       [40., 41., 42., 43., 44., 45., 46., 47.],
       [48., 49., 50., 51., 52., 53., 54., 55.],
       [56., 57., 58., 59., 60., 61., 62., 63.]])

In [66]:
B2

array([[ 0.,  1.,  2.,  3.,  8.,  9., 10., 11.],
       [16., 17., 18., 19., 24., 25., 26., 27.],
       [ 4.,  5.,  6.,  7., 12., 13., 14., 15.],
       [20., 21., 22., 23., 28., 29., 30., 31.],
       [32., 33., 34., 35., 40., 41., 42., 43.],
       [48., 49., 50., 51., 56., 57., 58., 59.],
       [36., 37., 38., 39., 44., 45., 46., 47.],
       [52., 53., 54., 55., 60., 61., 62., 63.]])

In [69]:
K

4

In [ ]:
A = np.arange(8*8).reshape(8, 8)
B =np.arange(8*8).reshape(8, 8)

In [71]:
fill_block_arrays(A, B, 8, cs2)

i: 0 j: 0 normalized_i: 0 normalized_j: 0 idx: 0
i: 0 j: 1 normalized_i: 0 normalized_j: 1 idx: 1
i: 0 j: 2 normalized_i: 0 normalized_j: 2 idx: 2
i: 0 j: 3 normalized_i: 0 normalized_j: 3 idx: 3
i: 0 j: 4 normalized_i: 2 normalized_j: 0 idx: 16
i: 0 j: 5 normalized_i: 2 normalized_j: 1 idx: 17
i: 0 j: 6 normalized_i: 2 normalized_j: 2 idx: 18
i: 0 j: 7 normalized_i: 2 normalized_j: 3 idx: 19
i: 1 j: 0 normalized_i: 0 normalized_j: 4 idx: 4
i: 1 j: 1 normalized_i: 0 normalized_j: 5 idx: 5
i: 1 j: 2 normalized_i: 0 normalized_j: 6 idx: 6
i: 1 j: 3 normalized_i: 0 normalized_j: 7 idx: 7
i: 1 j: 4 normalized_i: 2 normalized_j: 4 idx: 20
i: 1 j: 5 normalized_i: 2 normalized_j: 5 idx: 21
i: 1 j: 6 normalized_i: 2 normalized_j: 6 idx: 22
i: 1 j: 7 normalized_i: 2 normalized_j: 7 idx: 23
i: 2 j: 0 normalized_i: 1 normalized_j: 0 idx: 8
i: 2 j: 1 normalized_i: 1 normalized_j: 1 idx: 9
i: 2 j: 2 normalized_i: 1 normalized_j: 2 idx: 10
i: 2 j: 3 normalized_i: 1 normalized_j: 3 idx: 11
i: 2 j: 4 

In [72]:
A

array([[ 0.,  1.,  2.,  3.,  8.,  9., 10., 11.],
       [16., 17., 18., 19., 24., 25., 26., 27.],
       [ 4.,  5.,  6.,  7., 12., 13., 14., 15.],
       [20., 21., 22., 23., 28., 29., 30., 31.],
       [32., 33., 34., 35., 40., 41., 42., 43.],
       [48., 49., 50., 51., 56., 57., 58., 59.],
       [36., 37., 38., 39., 44., 45., 46., 47.],
       [52., 53., 54., 55., 60., 61., 62., 63.]])